<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/8%20-%20Chatbot/dante_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget https://www.liberliber.it/mediateca/libri/a/alighieri/la_divina_commedia/txt/la_divin.zip

--2019-04-20 16:34:03--  https://www.liberliber.it/mediateca/libri/a/alighieri/la_divina_commedia/txt/la_divin.zip
Resolving www.liberliber.it (www.liberliber.it)... 93.186.244.67
Connecting to www.liberliber.it (www.liberliber.it)|93.186.244.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232691 (227K) [application/zip]
Saving to: ‘la_divin.zip’

la_divin.zip        100%[===================>] 227.24K   462KB/s    in 0.5s    

2019-04-20 16:34:05 (462 KB/s) - ‘la_divin.zip’ saved [232691/232691]



In [4]:
!unzip la_divin.zip

Archive:  la_divin.zip
  inflating: la_divin.txt            


In [5]:
with open("la_divin.txt", encoding="latin-1") as divine_file:
  divine_txt = divine_file.read()
  
print(divine_txt[:100])

Dante Alighieri
La Divina Commedia

Questo e-book è stato realizzato anche grazie al sostegno di:
E-


In [6]:
start = divine_txt.find("Nel mezzo del cammin di nostra vita")
end = divine_txt.find("l'amor che move il sole e l'altre stelle.")

divine_txt = divine_txt[start:end]

print(divine_txt[:100])

Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura,
ché la diritta via era smarrit


In [0]:
import re

divine_txt = re.sub("[\(\[].*?[\)\]]", "", divine_txt)
divine_txt = re.sub("[A-Z][A-Z]+","",divine_txt)

divine_txt = divine_txt.replace("di Dante Alighieri","")

divine_txt = divine_txt.replace("\n"," ")
divine_txt = divine_txt.lower()

In [59]:
"""
import string

divine_txt = re.sub(r'[^\w\s]','',divine_txt)
divine_txt = divine_txt.replace("\n"," ")
divine_txt = divine_txt.lower()

print(divine_txt[:100])
"""

['Nel mezzo del cammin di nostra vita\nmi ritrovai per una selva oscura,\nché la diritta via era smarrita', "\nAhi quanto a dir qual era è cosa dura\nesta selva selvaggia e aspra e forte\nche nel pensier rinova la paura!\nTant' è amara che poco è più morte;\nma per trattar del ben ch'i' vi trovai,\ndirò de l'altre cose ch'i' v'ho scorte", "\nIo non so ben ridir com' i' v'intrai,\ntant' era pien di sonno a quel punto\nche la verace via abbandonai", "\nMa poi ch'i' fui al piè d'un colle giunto,\nlà dove terminava quella valle\nche m'avea di paura il cor compunto,\nguardai in alto e vidi le sue spalle\nvestite già de' raggi del pianeta\nche mena dritto altrui per ogne calle", "\nAllor fu la paura un poco queta,\nche nel lago del cor m'era durata\nla notte ch'i' passai con tanta pieta", "\nE come quei che con lena affannata,\nuscito fuor del pelago a la riva,\nsi volge a l'acqua perigliosa e guata,\ncosì l'animo mio, ch'ancor fuggiva,\nsi volse a retro a rimirar lo passo\nche non lasciò gi

In [9]:
!python -m spacy download it_core_news_sm

    100% |████████████████████████████████| 36.5MB 73.7MB/s 
  Running setup.py install for it-core-news-sm ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/it_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/it_core_news_sm

    You can now load the model via spacy.load('it_core_news_sm')



In [10]:
import spacy

nlp = spacy.load("it_core_news_sm")

def preprocess(text):

  tokens = nlp(text)
  tokens_filtered = [token.text for token in tokens]
  return tokens_filtered

tokens = preprocess(divine_txt)
tokens[:10]

['nel',
 'mezzo',
 'del',
 'cammin',
 'di',
 'nostra',
 'vita',
 'mi',
 'ritrovai',
 'per']

In [11]:
maxlen = 10

divine_sents = []

for i in range(maxlen, len(tokens)):
  divine_sents.append(tokens[i-maxlen:i])
  
print(divine_sents[0])
print(divine_sents[1])

['nel', 'mezzo', 'del', 'cammin', 'di', 'nostra', 'vita', 'mi', 'ritrovai', 'per']
['mezzo', 'del', 'cammin', 'di', 'nostra', 'vita', 'mi', 'ritrovai', 'per', 'una']


In [12]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(divine_sents)
divine_sents = tokenizer.texts_to_sequences(divine_sents)

divine_sents[0]

Using TensorFlow backend.


[44, 234, 31, 617, 8, 196, 164, 22, 13631, 11]

In [0]:
import numpy as np
divine_sents = np.array(divine_sents)

X = divine_sents[:,:-1]
Y = divine_sents[:,-1]

In [14]:
from keras.utils import to_categorical

vocab_size = len(tokenizer.word_counts)

Y = to_categorical(Y, num_classes=vocab_size+1)
Y.shape

(117816, 13632)

In [0]:
from keras import Model, Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(vocab_size+1, maxlen-1, input_length=maxlen-1))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50, activation="relu"))
model.add(Dense(vocab_size+1, activation="softmax"))

In [0]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
model.fit(X, Y, batch_size=128, epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
117816/117816 [==============================] - 45s 381us/step - loss: 6.9463 - acc: 0.0720
Epoch 2/10
117816/117816 [==============================] - 43s 361us/step - loss: 6.5358 - acc: 0.0749
Epoch 3/10
117816/117816 [==============================] - 43s 362us/step - loss: 6.3568 - acc: 0.0915
Epoch 4/10
117816/117816 [==============================] - 42s 360us/step - loss: 6.2007 - acc: 0.1089
Epoch 5/10
117816/117816 [==============================] - 41s 352us/step - loss: 6.0883 - acc: 0.1193
Epoch 6/10
117816/117816 [==============================] - 42s 358us/step - loss: 5.9765 - acc: 0.1284
Epoch 7/10
117816/117816 [==============================] - 42s 355us/step - loss: 5.8779 - acc: 0.1363
Epoch 8/10
117816/117816 [==============================] - 42s 359us/step - loss: 5.7904 - acc: 0.1438
Epoch 9/10
117816/117816 [==============================] - 41s 351us/step - loss: 5.7111 - acc: 0.1494
Epoch 10/10
1178

# PARCHEGGIO

In [70]:
import string

divine_txt = divine_txt.lower()
divine_sents = divine_txt.split(".")

divine_sents = [re.sub(r'[^\w\s]','', sent) for sent in divine_sents]
divine_sents = [sent.replace("\n"," ",) for sent in divine_sents]

print(divine_sents[0])

nel mezzo del cammin di nostra vita mi ritrovai per una selva oscura ché la diritta via era smarrita
